In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-12-22 03:39:01--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-12-22 03:39:01 (22.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
chars = sorted(list(set(text))) #get all the characters in the first 1000 characters
vocab_size = len(chars) # get the size of it

In [5]:
char_to_index = {char: index for index, char in enumerate(chars)}
index_to_char = {index: char for index, char in enumerate(chars)}


In [6]:
def encode_string(s):
    encoded_list = [char_to_index[char] for char in s]
    return encoded_list

In [7]:
def decode_list(l):
    decoded_string = ''.join([index_to_char[index] for index in l])
    return decoded_string

In [8]:
input_string = "hello"
encoded_result = encode_string(input_string)
decoded_result = decode_list(encoded_result)

print(encoded_result)
print(decoded_result)

[46, 43, 50, 50, 53]
hello


In [9]:
import torch
data = torch.tensor(encode_string(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [10]:
print(data[:1000])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [11]:
## split data in test vs. train

train = data[:int(0.9*len(data))]
val_data = data[int(0.9*len(data)):]

In [12]:
block_size = 8 # for training chunks of the data
train[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In the above code, we have something called "context", context is essentially taking the sequence of values previous and predicting what comes next. For example, in the context of `(18, 47, 56)`, 57 should come next. And this is only true for this specific context.



In [13]:
## to demonstrate this in python code
x = train[:block_size]
y = train[1:block_size+1] ## we offset by 1 because we want to predict what will come next

for i in range(block_size):
  context = x[:i+1]
  target = y[i]
  print(context, target)

tensor([18]) tensor(47)
tensor([18, 47]) tensor(56)
tensor([18, 47, 56]) tensor(57)
tensor([18, 47, 56, 57]) tensor(58)
tensor([18, 47, 56, 57, 58]) tensor(1)
tensor([18, 47, 56, 57, 58,  1]) tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47]) tensor(58)


In [14]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # get a random value
    x = torch.stack([data[i:i+block_size] for i in ix]) # the first block size (context)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # the target
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53



---


Now we're going to use a Bigram Language Model. For some context: A bigram language model is a simple statistical approach to language modeling in natural language processing. It calculates the probability of a word based on the occurrence of its preceding word. Mathematically, it is represented as P(w_n | w_(n-1)). Where w_n would be the current word and w_(n-1) is the previous. The probability of a specific bigram is calculated by dividing the count of that bigram by the count of its preceding word. While bigram models are simple, they capture some local language structure, although they may not handle long-range dependencies as well as more advanced models like trigrams, n-grams, or neural network-based models.

In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):


  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding = nn.Embedding(vocab_size, vocab_size) # table of size vocab_size x vocab_size

  def forward(self, idx, targets):

    logits = self.token_embedding(idx) # (B,T,C) (batch, time, channel) tensor (4,8,vocab_size)

    if targets is None:
          loss = None
    else:
          B, T, C = logits.shape
          logits = logits.view(B*T, C) # reshapes the logits tensor into a 2D tensor (flatten it)
          targets = targets.view(B*T) # flatten the target tensor
          loss = F.cross_entropy(logits, targets) #calcualte the loss between the 2, measures how well the logits match the targets

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # this is used to generate new sequences of tokens
    # also note that idx is (batch size, block size)

    for i in range(max_new_tokens):

            logits, loss = self(idx) # obtain the predictions for the given input sequence (idx)

            logits = logits[:, -1, :] # this becomes (batch size, channels), focus on the last step

            probs = F.softmax(logits, dim=-1) # convert into probabilities

            idx_next = torch.multinomial(probs, num_samples=1) # give us 1 sample (1 prediction)

            idx = torch.cat((idx, idx_next), dim=1) # whatever the prediction is, concatenate it with the current idx and use this to predict the next element

    return idx


The generate function above essentially takes a B x T array and it's goal is to generate new tokens. We input the number of tokens we want to generate and we essentially generate B x (T+new tokens). Each iteration adds 1 more until we reach the end.




In [16]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [17]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) # we generally use e-4 for learning rate, but since this is much smaller, we'll settle for -3

In [18]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.394822597503662


# Self-Attention Block

There are multiple ways to implement the self attention block for large language models. We can either average the past context, do matrix multiplication, add softmax, or the self-attention method used in the transformers paper (positional encodings). I'm going to focus on using the 4th method.

In [19]:
torch.manual_seed(42)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)

## method 1:

xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [20]:
## method 2 (matrix multiplication):

weights = torch.tril(torch.ones(T,T))
weights = weights / weights.sum(1, keepdim=True)
xbow2 = weights @ x ## matrix multiplication


torch.allclose(xbow, xbow2)


True

In [21]:
## method 3: using softmax

tril = torch.tril(torch.ones(T, T)) # this essentially creates a lower triangular matrix with the lower triangle being 1
wei = torch.zeros((T,T)) # all 0s matrix
wei = wei.masked_fill(tril == 0, float('-inf')) # mask all the 0s in the upper triangular matrix with -inf
wei = F.softmax(wei, dim=-1) # apply softmax (take the exponent of each term, then divide by the sum of that row)
xbow3 = wei @ x # matrix multiplication
torch.allclose(xbow, xbow3)


True

In [22]:
## method 4: self attention

torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)


k = key(x) # this becomes size (B, T, 16)
q = query(x) # this becomes (B,T,16)


# now we want all the keys to perform a dot product with the queries

wei = q @ k.transpose(-2, -1) # (B,T,16) x (B,16,T) => (B,T,T)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
output = wei @ v # v is the elements we aggregate, this makes it so the output will be 16 dimensional (the headsize)





In [23]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

#Final Model

In [33]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

We added dropout to prevent overfitting. Every time there is a forward or backward pass, it essentially shuts off a random subset of neurons (dropping them to 0). And it then trains without them. This essentially ends up training an ensemble of sub-networks. And at the end, all the sub-networks are merged.

In [91]:
C1 = 384

class Head(nn.Module):


  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(C1, head_size, bias=False)
    self.query = nn.Linear(C1, head_size, bias=False)
    self.value = nn.Linear(C1, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size))) # this creates the lower triangle matrix
    self.dropout = nn.Dropout(dropout)

  def forward(self, x): # copied from above
    B,T,C = x.shape
    k = self.key(x)
    q = self.query(x)

    wei = q @ k.transpose(-2, -1) * C ** 0.5
    wei = wei.masked_fill(self.tril[:T,:T]==0, float('-inf'))
    wei = F.softmax(wei,dim=-1)
    wei = self.dropout(wei)
    v = self.value(x)
    out = wei @ v
    return out




In [92]:
class MultiHeadAttention(nn.Module):

  def __init__(self,num_heads,head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for i in range(num_heads)]) # create multiple heads
    self.proj = nn.Linear(C1, C1)
    self.dropout = nn.Dropout(dropout)

  def forward(self,x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.proj(out)
    return out # concatenate all of the output

In [93]:
class FeedForward(nn.Module):

  def __init__(self,n_embd):
    super().__init__()
    self.net = nn.Sequential( # multiplication of 4 comes from the fact that the dimensionality of input is x, but the inner layer dimensionality is 4*x
        nn.Linear(n_embd, 4*n_embd), # linear layer with n_embd input and n_embd output
        nn.ReLU(),# activation function, allows for non linearity (we use ReLU to get over vanishing gradients) -> vanishing gradients is essentially when
        nn.Linear(n_embd * 4, n_embd),    #  the gradients are propagated backward from the output layer to the input layer, they can become very small (vanish) as they pass through many layers.
        nn.Dropout(dropout)          # When the gradients become extremely small, the weights of the early layers are updated only by tiny amounts, if at all.
    )



  def forward(self, x):
    return self.net(x)


In [94]:
class Block(nn.Module):

  def __init__(self, n_embd, n_head): ## n_embd is the embedding dimension, n_head are the number of heads
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)


  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x


# Training Model


In [95]:
class BigramLanguageModel_new(nn.Module):


  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, C1)
    self.position_embedding_table = nn.Embedding(block_size, C1)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(C1)
    self.lm_head = nn.Linear(C1, vocab_size)


  def forward(self, idx, targets=None):
    B,T = idx.shape
    tok_emb = self.token_embedding_table(idx) #
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb # (B,T,C) array, includes both the information about the tokens and their positions in the sequence
    x = self.blocks(x)
    x = self.ln_f(x)
    logits = self.lm_head(x)

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self,idx,max_new_tokens):
    for i in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :] # focus on the last time step
            probs = F.softmax(logits, dim=-1) # probabilities
            idx_next = torch.multinomial(probs, num_samples=1) # get the i +1th prediction
            idx = torch.cat((idx, idx_next), dim=1)  # concatenate the prediction with the current sequence
    return idx

In [96]:
model = BigramLanguageModel_new()

In [97]:
def get_batch1(split):
    # generate a small batch of data of inputs x and targets y
    data = train if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [98]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [99]:
m = model.to(device)

In [100]:
decode = lambda l: ''.join([itos[i] for i in l])

In [ ]:
for iter in range(max_iters):

    print(iter)

    # every once in a while evaluate the loss on train and val sets

    # sample a batch of data
    xb, yb = get_batch1('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))